# A food desert identification tool

Identification of "food deserts" is vitial for a number of groups such as: 
1. The grocer looking to build a store
2. Zoneing officals looking to locate a store to improve the local comunity 
3. Public health offices looking to improve conditions such as obecity amongst others.

The problems of "Food Deserts" is well documented:
* https://www.theguardian.com/society/2018/oct/12/more-than-a-million-uk-residents-live-in-food-deserts-says-study
* https://en.wikipedia.org/wiki/Food_desert
* https://www.smf.co.uk/publications/barriers-eating-healthily-uk/
* https://www.mdpi.com/1660-4601/14/11/1290
* https://journals.cdrs.columbia.edu/wp-content/uploads/sites/25/2016/10/120-237-1-PB.pdf

While usualy thought of as problems food deserts also represent opportunites for grocerers 

While searches for food deserts are identified in cities like New York and Chicago but the research to identify these areas are time consuming and often the result of many man years work.

In this project I will endevor to develope a tool using foursquare data, US census (2010) by zip code to identify food deserts, replicating the work below as a proof of concept.

https://www.usccr.gov/pubs/docs/IL-FoodDeserts-2011.pdf

Additionaly accepted definition of a food desert is lots of fast food with relatively few grocery stores but the population should also be included such that it is not just the ratio fast food to grocery stores but also the population that matters becuase only fast food in a airport with no fixed population is much less an issue than say a residential area where lack of access to affordable alternatives to fast food is important to community health.

## Data Souces:

* Foursquare for food store types and locations.

* Chicago zip codes
    * https://namecensus.com/igapo/zip_codes/metropolitan-areas/metro-zip/Chicago%20(IL)1.html

* Us population by zipcode
    * https://s3.amazonaws.com/SplitwiseBlogJB/2010+Census+Population+By+Zipcode+(ZCTA).csv

* lat long of us zipcodes
    * https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/
    * https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/New_York&lang=en&use_labels_for_header=true&csv_separator=%3B

* Folium for mapping and plotting

* Foursquare in particular categories
    * https://developer.foursquare.com/docs/build-with-foursquare/categories/
    * From the Resaurant group
        * Fast Food Restaurant
    * From the Food and Dining Shop group
        * Liquor Store
        * Convenience Store
        * Farmers Market
        * Organic Grocery
        * Supermarket
        * Grocery Store

## Data Usage:
The data will be combinded to generate a data set consiting of a count of the Facebook groups with Grocery stores and like as good things while Fast Food and Liquor Stores counting as bad each as a ratio of population then it is time for stats to see what can be learned and what models can be built


**Setup Testing and intial exploration of data**

In [443]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup 
import requests
import geocoder
import folium 


Collect and assemble the zip code data frames

In [444]:
URL = 'https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/New_York&lang=en&use_labels_for_header=true&csv_separator=%3B'
zipCodesDF = pd.read_csv(URL, sep=';')
zipCodesDF.shape

(43191, 8)

In [445]:
zipCodesDF.head(10)


,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,48834,Fenwick,MI,43.141649,-85.049480,-5,1,"43.141649,-85.04948"
1,55304,Andover,MN,45.254715,-93.286520,-6,1,"45.254715,-93.28652"
2,55422,Minneapolis,MN,45.014764,-93.339650,-6,1,"45.014764,-93.33965"
3,29079,Lydia,SC,34.296064,-80.113190,-5,1,"34.296064,-80.11319"
4,29390,Duncan,SC,34.888237,-81.969020,-5,1,"34.888237,-81.96902"
5,29446,Green Pond,SC,32.683083,-80.560740,-5,1,"32.683083,-80.56074"
6,30018,Jersey,GA,33.717989,-83.801451,-5,1,"33.717989,-83.801451"
7,30093,Norcross,GA,33.909952,-84.179400,-5,1,"33.909952,-84.1794"
8,30417,Claxton,GA,32.162077,-81.909840,-5,1,"32.162077,-81.90984"
9,30627,Carlton,GA,33.994139,-82.978190,-5,1,"33.994139,-82.97819"


In [450]:
URL = 'https://s3.amazonaws.com/SplitwiseBlogJB/2010+Census+Population+By+Zipcode+(ZCTA).csv'
zipcodePopulation = pd.read_csv(URL)

zipcodePopulation.head(10)

,Zip Code ZCTA,2010 Census Population
0,1001,16769
1,1002,29049
2,1003,10372
3,1005,5079
4,1007,14649
5,1008,1263
6,1009,741
7,1010,3609
8,1011,1370
9,1012,661


In [451]:
URL = 'https://namecensus.com/igapo/zip_codes/metropolitan-areas/metro-zip/Chicago%20(IL)1.html'
rawPage = requests.get(URL, 'html.parser')
soup = BeautifulSoup(rawPage.text, 'html.parser')
table = soup.find_all('table')[0]
chicagoZip = pd.read_html(str(table))[0]

chicagoZip.head(10)
foo = chicagoZip[0].str.split(' ', 1, expand=True)
foo.columns=["ZipCode", "Neighborhood"]
#foo.dtypes
foo["ZipCode"] = foo["ZipCode"].astype(int)
#foo.head(10)
completeDF = foo.merge(zipCodesDF, left_on='ZipCode', right_on='Zip', how='left')

completeDF = completeDF.merge(zipcodePopulation, left_on='ZipCode', right_on='Zip Code ZCTA', how='left')
completeDF = completeDF.drop(['Zip', 'geopoint', 'Daylight savings time flag', 'Timezone','Zip Code ZCTA'], axis=1).dropna()

reducedDF = completeDF.drop_duplicates( subset=['ZipCode'] )
reducedDF.head(30)


,ZipCode,Neighborhood,City,State,Latitude,Longitude,2010 Census Population
1,60002,Antioch,Antioch,IL,42.466170,-88.09995,24299.0
5,60004,Arlington Heights,Arlington Heights,IL,42.108428,-87.97723,50582.0
6,60005,Arlington Heights,Arlington Heights,IL,42.069327,-87.98464,29308.0
8,60007,Elk Grove Village,Elk Grove Village,IL,42.005978,-87.99847,33820.0
9,60008,Rolling Meadows,Rolling Meadows,IL,42.075060,-88.02508,22717.0
11,60010,Barrington,Barrington,IL,42.160791,-88.15231,44095.0
29,60012,Bull Valley,Crystal Lake,IL,42.265643,-88.31664,11120.0
33,60013,Cary,Cary,IL,42.217523,-88.24338,26872.0
36,60014,Crystal Lake,Crystal Lake,IL,42.226623,-88.33066,48550.0
40,60015,Bannockburn,Deerfield,IL,42.169325,-87.86556,26800.0


In [457]:
reducedDF.shape


(316, 7)

In [462]:
CLIENT_ID = 'VZAQKGZTYRUBC4NMWEYP3XS0BQBUJVQ1XSCAMJDCRRF2B0P2' # your Foursquare ID
CLIENT_SECRET = 'CJALW0S25ZR4K0DS31SYMB2TU3Z3VS0V3N4OS4RZTKTQ50CJ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
search_query = 'Grocery'
radius = 2000
postCodeList = []
jsonResult = []



toAdd = []
#reducedDF.iloc[0]
#lat = reducedDF.iloc[0]['Latitude']
#long = reducedDF.iloc[0]['Longitude']
#postcode =  reducedDF.iloc[0]['ZipCode']
for row in reducedDF.itertuples(index=True, name='Pandas'):
    lat = row.Latitude
    long = row.Longitude
    postCode = row.ZipCode
    postCodeList.append(postCode)
    searchURL = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
    result = requests.get(searchURL).json()
    jsonResult.append(result)
    toAdd = list(zip(postCodeList, jsonResult))

len(toAdd)
#df_temp = pd.DataFrame(toAdd, columns = ['PostalCode', 'jsonResult'])
#df_Farmers = df_with_geo.merge(df_temp, left_on='PostalCode', right_on='PostalCode')
#toAdd

0

In [459]:

toAdd[315][1]

IndexError: list index out of range

In [313]:
countsForZip = {}

for j in range(len(toAdd)) :
    #zipCode = toAdd[j][0]
    try :
        venues = json_normalize(toAdd[j][1]['response']['venues'])
        #print("vensus")
        vcDict = {}
        for i in range( len(venues) ) :
            for vc in venues.iloc[i]['categories'] :
                if vc['name'] in vcDict :
                    vcDict [vc['name']] += 1
                else :
                    vcDict [vc['name']] = 1
                    
        countsForZip[toAdd[j][0]] = vcDict
    except KeyError:
        None
        #print("no veneus")
    
#venues = json_normalize(toAdd[300][1]['response']['venues'])
#veneus
#len(venues)
#venues.iloc[2]['categories'][0]['name']
#venues.iloc[0]['categories'][0]['name']




# catList = get_categories (venues.iloc[0])

#venues.iloc[0]['categories'][0]['name']
#remember veneus is the venues at a location



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


316

In [314]:
countsForZip

{60002: {'Mobile Phone Shop': 1, 'Business Service': 1},
 60004: {'Grocery Store': 1,
  'Supermarket': 1,
  'Thrift / Vintage Store': 4,
  'Home Service': 1,
  'Business Service': 1,
  'Shipping Store': 1,
  'Gift Shop': 1,
  'Mobile Phone Shop': 1,
  'Department Store': 1},
 60005: {'Gourmet Shop': 1,
  'Hardware Store': 2,
  'Grocery Store': 1,
  'Pet Store': 1,
  'Business Service': 3,
  'Shipping Store': 1,
  'Mobile Phone Shop': 2,
  "Women's Store": 1,
  'Ice Cream Shop': 1},
 60007: {'Shipping Store': 2,
  'Mobile Phone Shop': 2,
  'Grocery Store': 1,
  'Business Service': 2,
  'Convenience Store': 1},
 60008: {'Business Service': 7, 'Bakery': 1, 'Shipping Store': 1},
 60010: {'Grocery Store': 1,
  'Business Service': 1,
  'Music Venue': 1,
  'Mobile Phone Shop': 1,
  'Shipping Store': 1},
 60012: {'Shipping Store': 1, 'Pizza Place': 1},
 60013: {'Discount Store': 1,
  'Convenience Store': 1,
  'Business Service': 2,
  'Hardware Store': 1},
 60014: {'Music Store': 1,
  'Hardware

In [442]:
countsForZipDF = pd.DataFrame({'ZipCode': [] })
for z in countsForZip :
    tempDF = pd.DataFrame.from_dict([countsForZip[z]])
    tempDF['ZipCode'] = z
    countsForZipDF = countsForZipDF.append(tempDF)

storeType = ['ZipCode','Grocery Store', 'Supermarket', 'Fast Food Restaurant', 'Liquor Store', 'Convenience Store']
countsForZipDF = countsForZipDF.fillna(0)
countsForZipDF = countsForZipDF[storeType]

countsForZipDF = countsForZipDF.merge(reducedDF, left_on='ZipCode', right_on='ZipCode', how='left')
countsForZipDF['Grocery Store RT'] = countsForZipDF['2010 Census Population'] / countsForZipDF['Grocery Store'] 
countsForZipDF['Supermarket RT'] = countsForZipDF['2010 Census Population'] / countsForZipDF['Supermarket'] 
countsForZipDF['Fast Food Restaurant RT'] = countsForZipDF['2010 Census Population'] / countsForZipDF['Fast Food Restaurant'] 
countsForZipDF['Liquor Store RT'] = countsForZipDF['2010 Census Population'] / countsForZipDF['Liquor Store'] 
countsForZipDF['Convenience Store RT'] = countsForZipDF['2010 Census Population'] / countsForZipDF['Convenience Store']
countsForZipDF 
        

,ZipCode,Grocery Store,Supermarket,Fast Food Restaurant,Liquor Store,Convenience Store,Neighborhood,City,State,Latitude,Longitude,2010 Census Population,Grocery Store RT,Supermarket RT,Fast Food Restaurant RT,Liquor Store RT,Convenience Store RT
0,60002.0,0.0,0.0,0.0,0.0,0.0,Antioch,Antioch,IL,42.466170,-88.09995,24299.0,inf,inf,inf,inf,inf
1,60004.0,1.0,1.0,0.0,0.0,0.0,Arlington Heights,Arlington Heights,IL,42.108428,-87.97723,50582.0,50582.0,50582.0,inf,inf,inf
2,60005.0,1.0,0.0,0.0,0.0,0.0,Arlington Heights,Arlington Heights,IL,42.069327,-87.98464,29308.0,29308.0,inf,inf,inf,inf
3,60007.0,1.0,0.0,0.0,0.0,1.0,Elk Grove Village,Elk Grove Village,IL,42.005978,-87.99847,33820.0,33820.0,inf,inf,inf,33820.00
4,60008.0,0.0,0.0,0.0,0.0,0.0,Rolling Meadows,Rolling Meadows,IL,42.075060,-88.02508,22717.0,inf,inf,inf,inf,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,60714.0,2.0,0.0,0.0,0.0,0.0,Niles,Niles,IL,42.032379,-87.81563,29931.0,14965.5,inf,inf,inf,inf
312,60803.0,0.0,0.0,0.0,0.0,0.0,Alsip,Alsip,IL,41.675861,-87.73189,22285.0,inf,inf,inf,inf,inf
313,60804.0,2.0,0.0,0.0,0.0,4.0,Chicago,Cicero,IL,41.844382,-87.75990,84573.0,42286.5,inf,inf,inf,21143.25
314,60805.0,1.0,0.0,0.0,0.0,0.0,Chicago,Evergreen Park,IL,41.719933,-87.70249,19852.0,19852.0,inf,inf,inf,inf


## Methodolgy

In [369]:
countsForZipDF.columns

Index(['Mobile Phone Shop_x', 'Business Service_x', 'ZipCode',
       'Grocery Store_x', 'Supermarket_x', 'Thrift / Vintage Store_x',
       'Home Service_x', 'Business Service_y', 'Shipping Store_x',
       'Gift Shop_x',
       ...
       'Grocery Store', 'Hardware Store_y', 'Thrift / Vintage Store',
       'Discount Store_y', 'Bank', 'Miscellaneous Shop', 'Business Service',
       'Cosmetics Shop_y', 'Mobile Phone Shop_y',
       'Residential Building (Apartment / Condo)_y'],
      dtype='object', length=2217)

## Results

## Results

## Discussion

The data for store type in the foursquare data is not suffcently clean. For example wireless stores appeared in the liquor store catagory. 

## Conclusion